# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
# from openai import OpenAI
# ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen-Image-Edit',
 '/deepseek-ai/DeepSeek-V3.1-Base',
 '/deepseek-ai/DeepSeek-V3.1',
 '/google/gemma-3-270m',
 '/ByteDance-Seed/Seed-OSS-36B-Instruct',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image-Edit',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/Qwen/Qwen-Image',
 '/spaces/lvwerra/jupyter-agent-2',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Granary',
 '/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1',
 '/datasets/allenai/WildChat-4.8M',
 '/datasets/nvidia/Nemotron-CC-v2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'brand / company resources', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers (job listings)',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'careers / join page', 'url': 'https://huggingface.co/join'},
  {'type': 'pricing', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise / solutions',
   'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog / news', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'},
  {'type': 'learning resources', 'url': 'https://huggingface.co/learn'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'community (Discord)',
   'url': 'https://huggingface.co/join/discord'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'social - GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'social - Twitt

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'home', 'url': 'https://huggingface.co/'}, {'type': 'about / company', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'products - models', 'url': 'https://huggingface.co/models'}, {'type': 'products - datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'products - spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'learn / tutorials', 'url': 'https://huggingface.co/learn'}, {'type': 'brand / media resources', 'url': 'https://huggingface.co/brand'}, {'type': 'blog / news', 'url': 'https://huggingface.co/blog'}, {'type': 'careers / jobs (external)', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'discord (community)', 'url': 'https://huggingface.co/join/di

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'products - models', 'url': 'https://huggingface.co/models'}, {'type': 'products - datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'products - spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'endpoints (product)', 'url': 'https://endpoints.huggingface.co'}, {'type': 'brand resources', 'url': 'https://huggingface.co/brand'}, {'type': 'careers / jobs', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn (education)', 'url': 'https://huggingface.co/learn'}, {'type': 'changelog', 'url': 'https://huggingface.co/changelog'}, {'type': 'chat (product)', 'url': 'htt

'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen-Image-Edit\nUpdated\n5 days ago\n•\n28.2k\n•\n1.1k\ndeepseek-ai/DeepSeek-V3.1-Base\nUpdated\n1 day ago\n•\n12.5k\n•\n863\ndeepseek-ai/DeepSeek-V3.1\nUpdated\n1 day ago\n•\n14.3k\n•\n460\ngoogle/gemma-3-270m\nUpdated\n9 days ago\n•\n63.8k\n•\n608\nByteDance-Seed/Seed-OSS-36B-Instruct\nUpdated\n1 day ago\n•\n3.13k\n•\n251\nBrowse 1M+ models\nSpaces\nRunning\n12.2k\n12.2k\nDeepSite 

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'brand assets', 'url': 'https://huggingface.co/brand'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page (external)', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'join / signup', 'url': 'https://huggingface.co/join'}, {'type': 'product - models', 'url': 'https://huggingface.co/models'}, {'type': 'product - datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'product - spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'product - chat', 'url': 'https://huggingface.co/chat'}, {'type': 'product - endpoints', 'url': 'https://endpoints.huggingface.co'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'learning resources', 'url': 'https://huggingface.co/learn'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'

# Hugging Face — The AI community building the future

A one-page brochure for prospective customers, investors and recruits

---

## Who we are
Hugging Face is the collaboration platform and open-source hub for machine learning. We connect a global community to discover, build and ship models, datasets and ML applications across text, image, video, audio and 3D — accelerating research and production AI.

Key numbers
- Browse 1M+ models
- 250k+ datasets
- 400k+ applications (Spaces)
- More than 50,000 organizations using Hugging Face
- Team size: ~210 members
- Community following: 55k+ (organization profile)

---

## What we offer
Products & platform capabilities
- Models Hub — host, discover and collaborate on public and private models.
- Datasets Hub — share and access datasets for every ML task.
- Spaces — deploy interactive ML applications and demos (Gradio / Streamlit).
- Compute — GPU instances to run and scale Spaces and training; GPU pricing from $0.60 / hour.
- Inference Endpoints & Text Generation Inference (TGI) — production-grade serving for language models.
- Enterprise & Team plans — SSO, regions, priority support, audit logs, resource groups, private dataset viewer. Team & Enterprise starting at $20 / user / month.
- HuggingChat and other community-facing apps for experimenting with models.

Why teams choose Hugging Face
- Host and collaborate on unlimited public models, datasets and apps.
- Fast path from research to production with inference endpoints and optimized compute.
- Built-in portfolio and profile to showcase work and recruit talent.
- Enterprise-grade controls and support for regulated or large organizations.

---

## Open-source foundation
Hugging Face builds and maintains core ML tooling used by research labs and production teams worldwide:
- Transformers — state-of-the-art models for PyTorch and beyond (148,691)
- Diffusers — diffusion models in PyTorch (30,449)
- Tokenizers — fast tokenizers (10,012)
- Safetensors — secure neural weight format (3,411)
- Hub Python client, PEFT, TRL, Accelerate, Transformers.js and more

(Counts shown reflect community activity / popularity on the Hub.)

---

## Customers & partners
Widely adopted across industry, research and non-profits. Representative organizations on the platform:
- Meta AI, Google, Microsoft, Amazon, Intel
- AI2 (Allen Institute for AI)
- Grammarly, Writer
More than 50,000 organizations rely on Hugging Face for model sharing, deployment and collaboration.

---

## Community & culture
Hugging Face is built around an open, collaborative culture:
- Community-first: a platform where researchers, engineers and hobbyists share models, datasets and apps publicly to advance the field.
- Open source at the core: major tooling and libraries are MIT/Apache-style open-source projects.
- Developer experience: focus on discoverability, reproducibility, and easy publishing (build your ML profile and portfolio).
- Rapid iteration: trending models and active Spaces demonstrate a fast-moving, experimental environment.
- Inclusive & knowledge-sharing: public docs, forum, blog and active Discord/GitHub communities.

Culture highlights
- Small, multidisciplinary teams working across research, product and infrastructure.
- Emphasis on collaboration, reproducibility and responsible AI practices.
- Fast growth: expanding team (~210 members) and global community contributions.

---

## Careers
Hugging Face is growing and hiring across engineering, research, product, developer relations and enterprise roles.

Why join
- Work at the intersection of open-source research and production ML.
- Contribute to widely used libraries and an active global community.
- Opportunity to impact how AI is built, shared and deployed.

Where to apply
- Visit the Jobs page on huggingface.co for current openings and role details.

---

## Get started / Contact
- Try the platform: Sign up at https://huggingface.co — explore models, datasets and Spaces.
- For enterprise inquiries: see Enterprise & Pricing pages for plans, SSO and dedicated support.
- Explore the open-source projects and the documentation to integrate Hugging Face tooling into your stack.

Hugging Face — a community and platform to discover, build and ship the future of AI.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'products - models', 'url': 'https://huggingface.co/models'}, {'type': 'products - datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'products - spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand / media assets', 'url': 'https://huggingface.co/brand'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'learn / tutorials', 'url': 'https://huggingface.co/learn'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'community - discord', 'url': 'https://huggingface.co/join/discord'}, {'typ

BadRequestError: Error code: 400 - {'error': {'message': 'Your organization must be verified to stream this model. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': 'stream', 'code': 'unsupported_value'}}

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>